# Práticas com MPI & Cluster

## Submeter o “Hello, World!” de MPI no cluster com slurm

In [ ]:
%%writefile hello.cpp
#include <mpi.h>
#include <iostream>
#include<unistd.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    if (size != 2) {
        std::cerr << "Este programa requer exatamente 2 processos." << std::endl;
        MPI_Abort(MPI_COMM_WORLD, 1);
    }

    int message;
    char hostname[256];
	  gethostname(hostname, sizeof(hostname));


    if (rank == 0) {

        message = 42;
        MPI_Send(&message, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        std::cout << hostname << ": Processo " << rank << " enviou " << message << " para P1." << std::endl;


        MPI_Recv(&message, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        std::cout << hostname << ": Processo " << rank << " recebeu " << message << " de P1." << std::endl;
    } else {
        MPI_Recv(&message, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        std::cout << hostname << ": Processo " << rank << " recebeu " << message << " de P0." << std::endl;

        message = 84;
        MPI_Send(&message, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
        std::cout << hostname << ": Processo " << rank << " enviou " << message << " para P0." << std::endl;
    }

    MPI_Finalize();
    return 0;
}

Arquivo do script slurm. Dependendo de como você estiver rodando, precisará usar o comando "--allow-run-as-root".

In [ ]:
%%writefile hello.slurm
#!/bin/bash
#SBATCH -N 2
#SBATCH --time=00:10:00
#SBATCH --partition=normal

mpirun --allow-run-as-root ./hello

Compilação do código

In [ ]:
mpic++ -o hello hello.cpp

Submetendo o job no cluster

In [ ]:
sbatch hello.slurm

## Submeter um job de MPI que sobrecarrega a comunicação no cluster, observando o uso da rede no Ganglia


In [ ]:
%%writefile broadcast.cpp
#include <mpi.h>
#include <iostream>
#include <string>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const int iterations = 10000000;

    for (int iter = 0; iter < iterations; ++iter) {
        if (rank == 0) {

            std::string message = "Iteration " + std::to_string(iter);;
            MPI_Bcast(&message, 20, MPI_CHAR, 0, MPI_COMM_WORLD);
            std::cout << "Iteração " << iter << ": Processo " << rank << " enviou \"" << message << "\" para todos os outros." << std::endl;
        } else {

            char received_message[20];
            MPI_Bcast(&received_message, 20, MPI_CHAR, 0, MPI_COMM_WORLD);
            std::cout << "Iteração " << iter << ": Processo " << rank << " recebeu \"" << received_message << "\" de P0." << std::endl;
        }


        MPI_Barrier(MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}


Compilando o código.

In [ ]:
mpic++ -o broadcast broadcast.cpp

Arquivo de configuração do slurm.

In [ ]:
%%writefile broadcast.slurm
#!/bin/bash
#SBATCH -n 2
#SBATCH --time=00:10:00
#SBATCH --partition=normal

# Executa o código MPI
mpirun --allow-run-as-root -n 2 ./broadcast

Submissão do job

In [ ]:
sbatch broadcast.slurm

## Round

Altere a configuração do seu cluster para ter mais cores antes de rodar o código abaixo! Qual é a saída obtida?

In [ ]:
%%writefile round.cpp
#include <mpi.h>
#include <sys/time.h>
#include <stdio.h>
#include <stdlib.h>

#define N (1024 * 1024 * 1)

int main(int argc, char* argv[])
{
  int size, rank;
  struct timeval start, end;
  char hostname[256];
  int hostname_len;

  MPI_Init(&argc, &argv);

  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);
  MPI_Get_processor_name(hostname, &hostname_len);

  char* buffer = (char*)malloc(sizeof(char) * N);


  if (rank == 0) {
    gettimeofday(&start, NULL);
    printf("Rank %d (running on '%s'): sending the message rank %d\n", rank, hostname, 1);
    MPI_Send(buffer, N, MPI_BYTE, 1, 1, MPI_COMM_WORLD);
    MPI_Recv(buffer, N, MPI_BYTE, size - 1, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("Rank %d (running on '%s'): received the message from rank %d\n", rank, hostname, size - 1);
    gettimeofday(&end, NULL);
    printf("%f\n", (end.tv_sec * 1000000.0 + end.tv_usec - start.tv_sec * 1000000.0 - start.tv_usec) / 1000000.0);

  } else {
    MPI_Recv(buffer, N, MPI_BYTE, rank - 1, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("Rank %d (running on '%s'): receive the message and sending it to rank %d\n", rank, hostname,
           (rank + 1) % size);
    MPI_Send(buffer, N, MPI_BYTE, (rank + 1) % size, 1, MPI_COMM_WORLD);
  }

  MPI_Finalize();
  return 0;
}

Agora você já sabe como compilar e executar o código!

## Soma de vetores

1. Crie um programa via C++/MPI que calcula a soma de um vetor de 10k números.
2. Submeta este programa no cluster via slurm:
 - Faça uma execução local (usando apenas um node)
 - Faça uma execução global (usando todos os nodes e recursos disponíveis).
 - Observe e compare os tempos de execução


In [ ]:
#include <mpi.h>
#include <iostream>
#include <vector>
#include <random>

// size of array
#define n 10000

// Temporary array for slave process
int a2[1000];

int main(int argc, char *argv[]) {
    int pid, np,
        elements_per_process,
        n_elements_received;
    // np -> no. of processes
    // pid -> process id

    MPI_Status status;

    // Initialize MPI
    MPI_Init(&argc, &argv);

    // find out process ID,
    // and how many processes were started
    MPI_Comm_rank(MPI_COMM_WORLD, &pid);
    MPI_Comm_size(MPI_COMM_WORLD, &np);

    // master process
    if (pid == 0) {
        int index, i;
        elements_per_process = n / np;

        // Initialize vector with a for loop
        std::vector<int> a;
        for (int i = 0; i < n; ++i) {
            a.push_back(i + 1); // Assuming you want elements from 1 to 10000
        }

        // check if more than 1 process is run
        if (np > 1) {
            // distributes the portion of array
            // to child processes to calculate
            // their partial sums
            for (i = 1; i < np - 1; i++) {
                index = i * elements_per_process;

                MPI_Send(&elements_per_process,
                         1, MPI_INT, i, 0,
                         MPI_COMM_WORLD);
                MPI_Send(&a[index],
                         elements_per_process,
                         MPI_INT, i, 0,
                         MPI_COMM_WORLD);
            }

            // last process adds remaining elements
            index = i * elements_per_process;
            int elements_left = n - index;

            MPI_Send(&elements_left,
                     1, MPI_INT,
                     i, 0,
                     MPI_COMM_WORLD);
            MPI_Send(&a[index],
                     elements_left,
                     MPI_INT, i, 0,
                     MPI_COMM_WORLD);
        }

        // master process adds its own sub-array
        int sum = 0;
        for (i = 0; i < elements_per_process; i++)
            sum += a[i];

        // collects partial sums from other processes
        int tmp;
        for (i = 1; i < np; i++) {
            MPI_Recv(&tmp, 1, MPI_INT,
                     MPI_ANY_SOURCE, 0,
                     MPI_COMM_WORLD,
                     &status);
            int sender = status.MPI_SOURCE;

            sum += tmp;
        }

        // prints the final sum of array
        std::cout << "Sum of array is: " << sum << std::endl;
    }
    // slave processes
    else {
        MPI_Recv(&n_elements_received,
                 1, MPI_INT, 0, 0,
                 MPI_COMM_WORLD,
                 &status);

        // stores the received array segment
        // in the local array a2
        MPI_Recv(&a2, n_elements_received,
                 MPI_INT, 0, 0,
                 MPI_COMM_WORLD,
                 &status);

        // calculates its partial sum
        int partial_sum = 0;
        for (int i = 0; i < n_elements_received; i++)
            partial_sum += a2[i];

        // sends the partial sum to the root process
        MPI_Send(&partial_sum, 1, MPI_INT,
                 0, 0, MPI_COMM_WORLD);
    }

    // cleans up all MPI state before exit of process
    MPI_Finalize();

    return 0;
}
